In [71]:
import re
import PyPDF2
import pdfplumber
import pandas as pd
import openai
from num2words import num2words
import os
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
import time

In [2]:
input_pdf = "D:\\Code\\CognitiveSearchGPT\\backend\\data\\2022_CMB_Report.pdf"  # 请将该路径替换为实际的输入PDF路径
output_pdf_prefix = "D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report"    # 输出文件的前缀

In [61]:
AZURE_OPENAI_API_KEY_SOUTH_CENTRAL_US=
AZURE_OPENAI_BASE=
openai.api_type = "azure"
openai.api_key = AZURE_OPENAI_API_KEY_SOUTH_CENTRAL_US
openai.api_base = AZURE_OPENAI_BASE
openai.api_version = "2022-12-01"

In [32]:
bboxes = []

def not_within_bboxes(obj):
    """Check if the object is in any of the table's bbox."""
    def obj_in_bbox(_bbox):
        """See https://github.com/jsvine/pdfplumber/blob/stable/pdfplumber/table.py#L404"""
        v_mid = (obj["top"] + obj["bottom"]) / 2
        h_mid = (obj["x0"] + obj["x1"]) / 2
        x0, top, x1, bottom = _bbox
        return (h_mid >= x0) and (h_mid < x1) and (v_mid >= top) and (v_mid < bottom)
    return not any(obj_in_bbox(__bbox) for __bbox in bboxes)

In [34]:
def curves_to_edges(cs):
    """See https://github.com/jsvine/pdfplumber/issues/127"""
    edges = []
    for c in cs:
        edges += pdfplumber.utils.rect_to_edges(c)
    return edges

In [86]:
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

In [35]:
with pdfplumber.open(input_pdf) as pdf:
        total_pages = len(pdf.pages)

        page_indices = []
        sections = []
        tables = []
        for i, page in enumerate(pdf.pages):
            page_text = page.extract_text()
            ts = {
                "vertical_strategy": "explicit",
                "horizontal_strategy": "explicit",
                "explicit_vertical_lines": curves_to_edges(page.curves + page.edges),
                "explicit_horizontal_lines": curves_to_edges(page.curves + page.edges),
                "intersection_y_tolerance": 10,
            }
            global bboxes
            bboxes = [table.bbox for table in page.find_tables(table_settings=ts)]
            page_text=page.filter(not_within_bboxes).extract_text()
            table_settings = {
                "vertical_strategy": "text",
                "horizontal_strategy": "text"
            }
            table = page.extract_tables(table_settings)
            if len(table) > 0:
                tables.append(table)
            # print(tables)
            rx = re.compile(r'''
                ^(?:Section\ )?(\d+\.+\d+)+
                [\s\S]*?
                (?=^(?:Section\ )?(\d+\.+\d+)+|\Z)

                ''', re.VERBOSE | re.MULTILINE)

            parts = [match.group(0) for match in rx.finditer(page_text)]
            # print(parts)
            for part in parts:
                if len(part) > 0:
                    sections.append(part)
            # if len(parts) > 0:
            #     sections.append(parts)
            # if find_chapters(page_text):
            #     page_indices.append(i)
        print(f"Total {len(sections)} parts.")
        print(f"Total {len(tables)} tables")

Total 234 parts.
Total 359 tables


In [59]:
table = tables[10]
df = pd.DataFrame(table[1::],columns=table[0])
df

Empty DataFrame
Columns: [(主要会计数据和财务指标, , , , （人民币百万元，特别注明除外）, , 经营业绩, , 营业收入, , 营业利润, , 利润总额, , 净利润, , 归属于本行股东的净利润, , 扣除非经常性损益后归属于本行股东的净利润, , 经营活动产生的现金流量净额, , 每股计（人民币元）, , 归属于本行普通股股东的基本每股收益(1), , 归属于本行普通股股东的稀释每股收益, , 扣除非经常性损益后归属于本行普通股股东的, , 基本每股收益, , 每股经营活动产生的现金流量净额, , 财务比率(%), , 归属于本行股东的平均总资产收益率, , 归属于本行普通股股东的平均净资产收益率(1), , 归属于本行普通股股东的加权平均净资产收益率, , 扣除非经常性损益后归属于本行普通股股东的, , 加权平均净资产收益率), (, , , , 2022年, , , , 344,783, , 165,156, , 165,113, , 139,294, , 138,012, , 137,551, , 570,143, , , , 5.26, , 5.26, , , , 5.25, , 22.61, , , , 1.42, , 17.06, , 17.06, , , , 17.00), (, , , , 2021年, , , , 331,253, , 148,019, , 148,173, , 120,834, , 119,922, , 119,430, , 182,048, , , , 4.61, , 4.61, , , , 4.59, , 7.22, , , , 1.36, , 16.96, , 16.96, , , , 16.89), (, , 本年比上年, , 增减(%), , , , 4.08, , 11.58, , 11.43, , 15.28, , 15.08, , 15.17, , 213.18, , , , 14.10, , 14.10, , , , 14.38, , 213.16, , , , 增加0.06个百分点, , 增加0.10个百分点, , 增加0.10个百分点, , , , 增加0.11个百分点), (, , , , 2020年, , , , 290,482, , 122,643, , 122,440, , 97,959, , 97,342, , 97,178, , 421,328, , , , 3.79, , 3.79, , , , 3.79, , 16.71, , , , 1.23, , 15.73, , 15.73, , , , 15.70)]
Index: []

In [45]:
sections[15]

'1.4 荣誉与奖项\n2022年，本公司在国内外机构组织的评选活动中获得诸多荣誉与奖项，其中：\n• 2022年2月，英国《银行家》杂志公布2022年“全球银行品牌价值500强”榜单，本公司凭借品牌价值\n'

In [63]:
section_embedding=get_embedding(sections[15],engine='text-embedding-ada-002')

In [64]:
section_embedding

[-0.024059414863586426,
 -0.026194244623184204,
 0.003161317901685834,
 -0.010798577219247818,
 -0.027608733624219894,
 0.021374505013227463,
 -0.011735022068023682,
 0.017340591177344322,
 -0.00364427431486547,
 -0.0135817164555192,
 4.4049305870430544e-05,
 0.017406076192855835,
 -0.023221198469400406,
 -0.017890669405460358,
 -0.014590194448828697,
 0.003238263539969921,
 0.010065138339996338,
 -0.026626450940966606,
 0.015454604290425777,
 -0.0004645385779440403,
 -0.016882190480828285,
 -0.012324392795562744,
 -0.026731226593255997,
 -0.006948023568838835,
 0.001956382766366005,
 0.017288202419877052,
 0.02831597998738289,
 -0.025631068274378777,
 0.029966216534376144,
 0.0010665969457477331,
 -0.006519092712551355,
 0.0022412450052797794,
 -0.019554004073143005,
 0.02821120247244835,
 -0.0014513249043375254,
 -0.018231194466352463,
 -0.02433445304632187,
 0.005310883279889822,
 0.006551835685968399,
 0.008755426853895187,
 0.01769421249628067,
 0.02705865539610386,
 0.01707864739

In [65]:
table_embedding = get_embedding(df.to_html(),engine='text-embedding-ada-002')

In [66]:
table_embedding

[-0.026928970590233803,
 0.0009448908385820687,
 0.00899864174425602,
 -0.028442135080695152,
 -0.009681574068963528,
 -0.0028807036578655243,
 -0.006685374770313501,
 -0.004763789940625429,
 -0.03101317584514618,
 -0.04780528321862221,
 0.019001595675945282,
 0.03470904752612114,
 -0.003946948796510696,
 0.024545401334762573,
 -0.005674366839230061,
 0.03749433904886246,
 0.005583978723734617,
 -0.025844313204288483,
 -0.008047892712056637,
 -0.042689982801675797,
 -0.017863374203443527,
 0.024264194071292877,
 -0.030209725722670555,
 0.017059924080967903,
 -0.024317758157849312,
 0.01626986637711525,
 0.006307083647698164,
 -0.013953250832855701,
 -0.01941671222448349,
 0.00694314856082201,
 0.010257380083203316,
 -0.012024970725178719,
 -0.010384593158960342,
 -0.017849983647465706,
 -0.0004699346318375319,
 0.006528032477945089,
 0.007746598683297634,
 0.022376086562871933,
 0.037708595395088196,
 -0.005480199586600065,
 -0.0009934325935319066,
 0.0010009650141000748,
 -0.002179358

In [67]:
tables_html=[]
for table in tables:
    df = pd.DataFrame(table[1::],columns=table[0])
    tables_html.append(df.to_html())
            
df_pdf = pd.DataFrame({'text':sections+tables_html})

In [70]:
len(df_pdf)

593

In [87]:
# Split dataframes to meet Azure OpenAI rate/token limit, may take up to 20mins
df_split = np.array_split(df_pdf,10)
for index, df in enumerate(df_split):
    if not os.path.exists(f"D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{index}.csv"):
        df['text']= df["text"].apply(lambda x : normalize_text(x))
        df['ada_v2'] = df['text'].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002'))
        df.to_csv(f'D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{index}.csv',mode='a',header=False,encoding='utf-8')
        time.sleep(60)
# df_pdf['ada_v2'] = df_pdf["text"].apply(lambda x : get_embedding(x, engine = 'text-embedding-ada-002')) # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model

In [106]:
df = df_split[8]
df.head(4)

,text
475,"<table border=""1"" class=""dataframe"">\n <thead..."
476,"<table border=""1"" class=""dataframe"">\n <thead..."
477,"<table border=""1"" class=""dataframe"">\n <thead..."
478,"<table border=""1"" class=""dataframe"">\n <thead..."


In [122]:
dfs = []
for i in range(10):
    df = pd.read_csv(f"D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{i}.csv", names= ['id','text','ada_v2'], encoding="utf-8")
    dfs.append(df)
df_all = pd.concat(dfs)
df_all

,id,text,ada_v2
0,0,1.2万人。\n向善，我们积极服务实体经济、履行企业社会责任。我们善待社会，把握国家扩大内需...,"[-0.02263690158724785, -0.016115764155983925, ..."
1,1,141.68亿元，同比增长6.60%；加强科技人才队伍建设，年末科技人才达1.08万人，较上...,"[-0.008546697907149792, -0.012597580440342426,..."
2,2,1.1 公司基本情况\n,"[-0.001001556753180921, -0.019434738904237747,..."
3,3,1.1.1 法定中文名称：招商银行股份有限公司（简称：招商银行）\n法定英文名称：China...,"[-0.006726536899805069, -0.008930496871471405,..."
4,4,1.1.2 法定代表人：缪建民\n授权代表：王良、何咏紫\n董事会秘书：王良\n联席公司秘书...,"[0.0029503011610358953, -0.007695871405303478,..."
...,...,...,...
54,588,"<table border=""1"" class=""dataframe""> <thead> <...","[-0.02490413375198841, -0.007793544791638851, ..."
55,589,"<table border=""1"" class=""dataframe""> <thead> <...","[-0.009898689575493336, -0.013569368980824947,..."
56,590,"<table border=""1"" class=""dataframe""> <thead> <...","[-0.012350639328360558, -0.012840904295444489,..."
57,591,"<table border=""1"" class=""dataframe""> <thead> <...","[-0.01899154670536518, -0.010021443478763103, ..."


In [124]:
df = pd.read_csv(f"D:\\Code\\CognitiveSearchGPT\\backend\\data\\CMB\\CMB_Report_embedding_{0}.csv",names= ['id','text','ada_v2'], encoding="utf-8")
df.head(5)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      60 non-null     int64 
 1   text    60 non-null     object
 2   ada_v2  60 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.5+ KB


In [125]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine="text-embedding-ada-002" # engine should be set to the deployment name you chose when you deployed the text-embedding-ada-002 (Version 2) model
    )
    df["similarities"] = df["ada_v2"].apply(lambda x: cosine_similarity(eval(x), embedding))

    res = (
            df.sort_values("similarities", ascending=False)
            .head(top_n)
        )
    return res


res = search_docs(df_all, "2022年总资产是多少?", top_n=4)

In [126]:
res

,id,text,ada_v2,similarities
37,37,"3.3.1 资产\n截至报告期末，本集团资产总额101,389.12亿元，较上年末增长9.6...","[-0.023433517664670944, -0.010127827525138855,...",0.859428
21,21,3.1 总体经营情况分析\n2022年，本集团坚持“质量、效益、规模”动态均衡发展理念，稳健...,"[-0.025321876630187035, -0.008960048668086529,...",0.854956
51,171,"5.12.1 2022年度利润分配方案\n按照经审计的本公司2022年度净利润1,284.8...","[-0.011170134879648685, -0.017800748348236084,...",0.853348
39,39,3.3.1.2 投资证券及其他金融资产\n本集团投资证券及其他金融资产包括以人民币和外币计价...,"[-0.0044101267121732235, -0.016428861767053604...",0.846161


In [133]:
question = "给我财务业绩摘要表"
res = search_docs(df_all, question, top_n=4)

context = res['text'].values.tolist()
context_str = "\n".join(context)
prompt = '''<|im_start|>系统助理帮助公司员工解决他们的问题，回答要简短。
仅回答以下来源列表中列出的事实。如果下面没有足够的信息，请说您不知道。不要生成不使用以下来源的答案。不要使用年代久远的来源信息。对于表格形式的数据，请以HTML表格形式输出，不要使用Markdown表格.
来源:
{sources}
<|im_end|>
<|im_start|>
user: {question}
<|im_end|>
<|im_start|>assistant:
'''
completion = openai.Completion.create(
            engine='gpt-35-turbo', 
            prompt=prompt.format(sources=context_str,question=question), 
            temperature=0.0, 
            max_tokens=2000, 
            n=1, 
            stop=["<|im_end|>", "<|im_start|>"])

result = completion.choices[0].text
print(result)

以下是财务业绩摘要表：

| 损益项目 | 2022年（人民币百万元） | 2021年（人民币百万元） | 变动额（人民币百万元） |
| --- | --- | --- | --- |
| 净利息收入 | 203,919 | 14,316 | 189,603 |
| 净手续费及佣金收入 | 94,447 | (172) | 94,619 |
| 其他净收入 | 32,887 | (614) | 33,501 |
| 业务及管理费 | (109,727) | (3,648) | (106,079) |
| 税金及附加 | (2,772) | (233) | (2,539) |
| 信用减值损失 | (65,962) | 9,211 | (75,173) |
| 其他资产减值损失 | (393) | (422) | 29 |
| 其他业务成本 | (4,380) | (1,301) | (3,079) |
| 营业外收支净额 | 154 | (197) | 351 |
| 利润总额 | 148,173 | 16,940 | 131,233 |
| 所得税费用 | (27,339) | 1,520 | (28,859) |
| 净利润 | 120,834 | 18,460 | 102,374 |
| 归属于本行股东的净利润 | 119,922 | 18,090 | 101,832 |
